In [1]:
import QUANTAXIS as QA
import datetime
import warnings
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')
import quantaxis_ext
from talib import MA_Type
import talib
import jupyter_helper
import calculator
import os
from tqdm import tqdm_notebook
import simulate
from pyecharts import Line
#加载 seaborn，并且设置默认使用 seaborn
import seaborn as sns
sns.set()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
%config InlineBackend.figure_format = 'svg'

In [2]:
zs_code = jupyter_helper.get_zs_code()
'指数代码:{0}'.format(zs_code)

start_date, end_date = jupyter_helper.get_start_end_date()
start_str = QA.QAUtil.QADate.QA_util_datetime_to_strdate(start_date)
end_str = QA.QAUtil.QADate.QA_util_datetime_to_strdate(end_date)
'开始日期:{0}'.format(start_date)
'结束日期:{0}'.format(end_date)

'指数代码:399300'

'开始日期:2005-04-08 00:00:00'

'结束日期:2019-01-18 00:00:00'

### 读取股票列表

In [3]:
lst = QA.QA_fetch_stock_list_adv()

In [4]:
### 读取或创建年度分析数据

In [5]:
def get_year_stats(start,end,lst=None):
    '''
    读取指定股票的年度分析数据。
    
    首先会尝试读取 datas 目录下的 stock_stas_year.h5 文件。如果文件不存在或有部分内容不存在，则读取部分内容。

    部分内容以单支股票为单位。加入只是有部分年份不存在，又或者上次生成文件时2018年，但现在时2020年，缺少2019年的数据时，也不会重新生成。
    
    Args:
        lst: 股票代码列表。为None时，自动获取所有列表。
    '''
    filename = os.path.join(os.getcwd(), 'datas', 'stock_stats_year.h5')
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    if lst is None or len(lst)==0:
        lst = QA.QA_fetch_stock_list_adv().code.values

    stats_by_year = pd.DataFrame()
    if os.path.exists(filename):
        stats_by_year = pd.read_hdf(filename, key='year', mode='r').astype('float64')
    has_new = False
    for stock in tqdm_notebook(lst):
        if stats_by_year.empty:
            stats_by_year = calculator.calc_perf_stats_by_year(stock, zs_code,start, end)
            has_new=True
        elif stock not in stats_by_year.index.unique(0).values:
            d = calculator.calc_perf_stats_by_year(stock, zs_code,start, end)
            if d.empty:
                continue
            stats_by_year = stats_by_year.append(d)
            has_new=True
    if has_new:
        stats_by_year.to_hdf(filename, key='year', mode='w')
        print('{0} updated.'.format(filename))
    return stats_by_year

lst_year_stats=get_year_stats(start_str,end_str,lst.code.values)

QA Error QA_fetch_stock_day_adv parameter code=002946 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=002947 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300755 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300758 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300759 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=600928 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601298 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601615 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None


### 所有上市日期在指数前的列表

In [6]:
valid_lst=[]
for code in set(lst_year_stats.index.get_level_values(0)):
    if lst_year_stats.loc[code].dropna().shape[0]==end_date.year-start_date.year:
        valid_lst.append(code)

### 取年度分析数据中的夏普比率

In [8]:
year_stats_describe=lst_year_stats.loc[valid_lst].groupby('code').agg(['max','min','mean','std'])
ysd_sr=year_stats_describe['Sharpe ratio']

**按照年度夏普比率均值倒序排序，均值越大越好
按照年度夏普比率标准差顺序排序，标准差越小越好**

In [9]:
def _sort_buy_std_and_mean(ysd_sr):
    '''根据年度数据的describe中的std和mean综合排序
    std按照从小到大，
    mean按照从大到小
    '''
    std_df=pd.DataFrame(ysd_sr.sort_values('std').reset_index()['code'])
    std_df['order']=std_df.index
    std_df=std_df.set_index('code')
    mean_df=pd.DataFrame(ysd_sr.sort_values('mean',ascending=False).reset_index()['code'])
    mean_df['order']=mean_df.index
    mean_df=mean_df.set_index('code')
    df=std_df.join(mean_df,lsuffix='_std',rsuffix='_mean')
    df['score']=df['order_std']+df['order_mean']
    return df
# 按照总得分顺序排序后的股票列表（总得分越小越好）
sorted_list=_sort_buy_std_and_mean(ysd_sr).sort_values('score').index.values

### 模拟每支股票，从开始到结束日期的表现

每支股票初始资金10000

In [11]:
class QA_Performance_Ext(QA.QA_Performance):
    '''此方法包含在finance_ai_py\quantaxis_ext.py'''

    @property
    def pnl_fifo_qfq(self):
        pnl = super(QA_Performance_Ext, self).pnl_fifo
        #         pnl['buy_price_qfq'] = pnl['buy_price']
        pnl = pnl.reset_index()
        for idx in pnl.index:
            s = QA.QAUtil.QADate.QA_util_datetime_to_strdate(
                pnl.loc[idx, 'buy_date'])
            e = QA.QAUtil.QADate.QA_util_datetime_to_strdate(
                pnl.loc[idx, 'sell_date'])
            c = pnl.loc[idx, 'code']
            new_value = QA.QA_fetch_stock_day_adv(c, s, e).to_qfq()
            pnl.loc[idx, 'buy_price_qfq'] = np.round(
                new_value.select_day(s).close[0], 4)
        pnl = pnl.assign(
            pnl_money_qfq=(pnl.sell_price - pnl.buy_price_qfq) * pnl.amount)
        return pnl.set_index('code')

In [20]:
import json
d={}
start_date, end_date = jupyter_helper.get_start_end_date()
start_str = QA.QAUtil.QADate.QA_util_datetime_to_strdate(start_date)
end_str = QA.QAUtil.QADate.QA_util_datetime_to_strdate(end_date)
for stock in tqdm_notebook(valid_lst):
    filename = os.path.join(os.getcwd(), 'datas', 'simulate','account','2005-2019','{0}.h5'.format(stock))
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    account = None
    try:
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                account = QA.QA_Account().from_message(json.load(f))
    except Exception as e:
        print(str(e))
        account=None
    if account is None:
        account=simulate._bbands_single(stock,start_str,end_str)
        try:
            with open(filename, 'w') as f:
                json.dump(account.message, f)
        except Exception as e:
            print(str(e))
            con
    d[stock]=account

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

...

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal

<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal
<class 'QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day'>
receive deal